Importing Necessary Libraries

In [ ]:
#!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv
#!pip install torch-geometric
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import seaborn as sns
import time
import random
import pandas as pd
import yaml

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


temperature=450

interval=200



Check wheather CUDA is available

In [ ]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Extracting Node feature Matrix and Edge List

In [ ]:
scaler = StandardScaler()

with open("config.yml", "r") as file:
    config = yaml.safe_load(file)

graph_directory=config['directories']['graph_directory']
lmp_file_directory=config['directories']['lmp_file_directory']


all_node_features = np.vstack([np.load(graph_directory+f'\\T={temperature}K\\feature.{temperature}K.{i}.npy') for i in range(interval, 60000 + 1, interval)])
scaler.fit(all_node_features)

graph_list = []
label = np.genfromtxt(lmp_file_directory+f'\\T={temperature}K\\Potential Energy vs step T_{temperature}.txt')



for i in range(interval, 60000 + 1, interval):
    node_features = np.load(graph_directory+f'\\T={temperature}K\\feature.{temperature}K.{i}.npy')
    node_features = scaler.transform(node_features)
    edge_indices = np.load(graph_directory+f'\\T={temperature}K\\edge.{temperature}K.{i}.npy').transpose()

    #edge_indices = np.concatenate((edge_indices, np.flip(edge_indices, axis=0)), axis=1)
    
    #adding self loops
    self_loop=np.arange(0, 13500).reshape(1, -1)
    self_loop_add=np.concatenate((self_loop,self_loop),axis=0)
   # print(self_loop.shape,self_loop_add.shape)
    edge_indices=np.concatenate((edge_indices,self_loop_add),axis=1)
    

    graph_label = torch.tensor(label[i, 1] / 13500, dtype=torch.float)

    node_features = torch.tensor(node_features, dtype=torch.float)
    edge_indices = torch.tensor(edge_indices, dtype=torch.long)
    
    graph = Data(x=node_features, edge_index=edge_indices, y=graph_label)
    graph.graph_name = i
    graph_list.append(graph)

print(graph_list[0].x.shape, graph_list[0].edge_index.shape)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def worker_init_fn(worker_id):
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)
batch_size = 1
loader = DataLoader(graph_list, batch_size=batch_size, shuffle=True,worker_init_fn=worker_init_fn)
print(loader)


Defining Model

In [ ]:
class GCNModel(torch.nn.Module):
    def __init__(self, in_channel, hidden_channel,hidden_layers):
        super().__init__()
        self.conv1 = GCNConv(in_channel, hidden_channel,aggr='sum')
        self.hidden_conv=nn.ModuleList()
        for _ in range (hidden_layers):
            self.hidden_conv.append(GCNConv(hidden_channel,hidden_channel,aggr='sum'))
        
        self.conv4 = GCNConv(hidden_channel, 1,aggr='sum')
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(13500, 1)

    def forward(self, x, edge_index):
        x=self.relu(self.conv1(x,edge_index))
        x=self.dropout(x)
        
        for h in self.hidden_conv:
            x=self.relu(h(x,edge_index))
            x=self.dropout(x)
            
        x = self.conv4(x, edge_index)
        x = torch.squeeze(x, 1)
        x = self.fc(x)
        return x

def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index)
        loss = criterion(output, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def predict(loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_names = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            output = model(batch.x, batch.edge_index)
            all_preds.append(output.cpu())
            all_labels.append(batch.y.cpu())
            all_names.extend([int(name) for name in batch.graph_name])
    return torch.cat(all_preds), torch.cat(all_labels), all_names



Training Model

In [ ]:
in_channel = graph_list[0].x.shape[1]
hidden_channel = 100  # Increased model complexity #previous 64
epochs = 800 #previous 150
lr = 0.0001
best_loss=1e3

model = GCNModel(in_channel, hidden_channel,1).to(device)
#model.apply(init_weights)  # Apply weight initialization
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()
#scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
start_time = time.time()
losses = []
for epoch in range(epochs):
    loss = train()
    #scheduler.step()
    losses.append(loss)
    if loss<best_loss:
        best_epoch=epoch+1
        best_model_state = model.state_dict().copy()
        best_loss=loss
        print(f'Best epoch = {best_epoch}')
    
    print(f'Epoch = {epoch + 1}/{epochs} Loss = {loss}')
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to run the code: {elapsed_time/60:.6f} minutes")
plt.plot(range(epochs), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

model.load_state_dict(best_model_state)

preds, labels, names = predict(loader)


preds = preds.numpy()
labels = labels.numpy()


plt.figure(figsize=(10, 5))
plt.plot(names, labels, label='Actual')
plt.plot(names, preds, label='Predicted', linestyle='dashed')
plt.xlabel('MC Steps')
plt.ylabel('Normalized Potential Energy')
plt.title('Actual vs Predicted Potential Energy')
plt.legend()
plt.show()

Make Predictions

In [ ]:

def predict(loader):
    model.eval()
    all_preds = []
    all_labels = []
    all_names = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            output = model(batch.x, batch.edge_index)
            all_preds.append(output.cpu())
            all_labels.append(batch.y.cpu())
            all_names.extend([int(name) for name in batch.graph_name])
    return torch.cat(all_preds), torch.cat(all_labels), all_names


preds, labels, names = predict(loader)


preds = preds.numpy()
labels = labels.numpy()


plt.figure(figsize=(10, 5))
plt.scatter(names, labels, label='Actual')
plt.scatter(names, preds, label='Predicted', linestyle='dashed')
plt.xlabel('MC Steps')
plt.ylabel('Normalized Potential Energy')
plt.title('Actual vs Predicted Potential Energy')
plt.legend()
plt.show()


In [ ]:


preds = preds.flatten()
labels = labels.flatten()


df = pd.DataFrame({
    'Index': names,
    'Actual': labels,
    'Predicted': preds
})


df = df.sort_values(by='Index')

df.to_csv(f'predictions_vs_actuals_{temperature}.csv', index=False)



In [ ]:




data = pd.read_csv(f'case_study_1_predictions_vs_actuals_{temperature}.csv')

plt.figure(figsize=(8, 4))
plt.plot(data['Index'], data['Actual'], label='Actual', marker='o')
plt.plot(data['Index'], data['Predicted'], label='Predicted', linestyle='dashed', marker='x')
plt.xlabel('MC Steps')
plt.ylabel('Normalized Potential Energy')
plt.title(f'Actual vs Predicted Potential Energy at {temperature}K')
plt.legend()
plt.savefig(f'case_study_1-{temperature}K')
plt.show()


